
### **Generating Abstract Cartoon Images with Generative Adversarial Networks and Neural Style Transfer**

Jiahui Shao, Student ID: 210934641



## Logistics Code:

In [1]:
#@title Imports

import tensorflow as tf
import keras
import numpy as np
from tqdm.autonotebook import tqdm
from IPython import display
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os
import imageio as iio
import random 
import tensorflow_hub as hub
# from tensorflow.keras import applications
# from tensorflow.keras.utils import plot_model
# from keras import Model
# from tensorflow.keras.optimizers import SGD
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  


In [2]:
 # @title Load GAN dataset

# # Load cartoon headshot datasets from Google Drive into 'train_images' and 'test_images' variables
# training_path = '/content/gdrive/My Drive/ECS7022P/datasets/training_set/'
# test_path = '/content/gdrive/My Drive/ECS7022P/datasets/test_set/'

# data_training = []
# data_test = []

# for filename in os.listdir(training_path):
#   image = Image.open(training_path + filename)
#   data_training.append(np.asarray(image))

# for filename in os.listdir(test_path):
#   image = Image.open(test_path + filename)
#   data_test.append(np.asarray(image))

# train_images = np.array(data_training)
# test_images = np.array(data_test)
# random.shuffle(train_images)  # shuffle the training set
# random.shuffle(test_images)  # shuffle the test set 

# print('train_images.shape: {}'.format(train_images.shape))  # 80% datasets for training
# print('test_images.shape: {}'.format(test_images.shape))  # 20% datasets for test

In [3]:
#@title Displaying the training images of the GAN phase 

# Helper function for displaying images
def show_images(images):
    if len(images.shape) > 3 and images.shape[3] == 1:
        if tf.is_tensor(images):
            images = images.numpy()
        images = np.squeeze(images, axis=3)

    num_images = len(images)
    fig, cells = plt.subplots(ncols=num_images, nrows=1, figsize=(2 * num_images, 2))
    for cell_num in range(num_images):
        cells[cell_num].matshow(images[cell_num])
        cells[cell_num].axis('off')
    plt.show()

# # Show some of the training images
# n_images =  16#@param{type: "integer"}
# images_display = train_images
# if n_images > 0:
#     images_display = images_display[:n_images]

# # Display!
# show_images(images_display)

In [4]:
#@title Data pre-processing

# # default: 512
# BATCH_SIZE = 256 # @param {type:"slider", min:128, max:1024, step:128}

# N_CHANNELS = (train_images.shape[-1] if len(train_images.shape)>3 else 1)
# TRAIN_BUF = len(train_images)
# IMG_SIZE = len(train_images[0])
# DIMS = (IMG_SIZE, IMG_SIZE, N_CHANNELS)
# N_TRAIN_BATCHES = int(TRAIN_BUF/BATCH_SIZE)

# N_CHANNELS_TEST = (test_images.shape[-1] if len(test_images.shape)>3 else 1)
# TEST_BUF = len(test_images)
# IMG_SIZE_TEST = len(test_images[0])
# N_TEST_BATCHES = int(TEST_BUF/BATCH_SIZE)

# print("Batch size: {} \nTraining set size: {}\
#        \nTest set size: {}\nImage Dimensions: {}\
#        \n# Train batches: {}\n# Test batches: {}".format(
#             BATCH_SIZE, TRAIN_BUF, TEST_BUF, DIMS, N_TRAIN_BATCHES, N_TEST_BATCHES
#         )
# )

BATCH_SIZE = 256

N_CHANNELS = 3
TRAIN_BUF = 4000
IMG_SIZE = 64
DIMS = (64,64,3)
N_TRAIN_BATCHES = 15

N_CHANNELS_TEST = 3
TEST_BUF = 1000
IMG_SIZE_TEST = 64
N_TEST_BATCHES = 3

In [5]:
#@title Normalise data

# # Normalise the data between 0 and 1. First apply the .astype("float32") function to the training dataset. Next, given that the colour layers go between 0 and 255, divide by 255.0 the 'train_images' and 'test_images' variable.
# train_images = train_images.reshape(train_images.shape[0], IMG_SIZE, IMG_SIZE, N_CHANNELS).astype("float32")
# train_images = train_images/255.0  # (train_images - 127.5) / 127.5
# test_images = test_images.reshape(test_images.shape[0], IMG_SIZE_TEST, IMG_SIZE_TEST, N_CHANNELS_TEST).astype("float32")
# test_images = test_images/255.0  # (train_images - 127.5) / 127.5

In [6]:
#@title Batch datasets

# # create batch size for training 
# train_dataset = (
#     tf.data.Dataset.from_tensor_slices(train_images)
#     .shuffle(TRAIN_BUF)
#     .batch(BATCH_SIZE)
# )

# test_dataset = (
#     tf.data.Dataset.from_tensor_slices(test_images)
#     .shuffle(TEST_BUF)
#     .batch(BATCH_SIZE)
# )

In [7]:
#@title Set up generator network structure

# default: "relu"
activation_l0 = "relu" #@param{type: "string"}
# default: "relu"
activation_l2 = "relu" #@param{type: "string"}
# default: "relu"
activation_l3 = "relu" #@param{type: "string"}
# default: "sigmoid"
activation_l4 = "sigmoid" #@param{type: "string"}

n_stride = 2  # factor to scale image up by in each convolution layer
n_layers_stride = 2  # number of layers scaling the image up
n_smallest_dim = IMG_SIZE//(n_stride*n_layers_stride)
seed_size = IMG_SIZE * (2 ** (n_layers_stride-1)) * N_CHANNELS

# Generator structure, 5 layers: Dense, Reshape, 3xConvolutionTranspose
generator = [
    tf.keras.layers.Dense(units=n_smallest_dim * n_smallest_dim * seed_size, activation=activation_l0),
    tf.keras.layers.Reshape(target_shape=(n_smallest_dim, n_smallest_dim, seed_size)),
    tf.keras.layers.Conv2DTranspose(
        filters=seed_size, kernel_size=3, strides=(n_stride, n_stride), padding="SAME", activation=activation_l2
    ),
    tf.keras.layers.Conv2DTranspose(
        filters=seed_size/2, kernel_size=3, strides=(n_stride, n_stride), padding="SAME", activation=activation_l3
    ),
    tf.keras.layers.Conv2DTranspose(
        filters=N_CHANNELS, kernel_size=3, strides=(1, 1), padding="SAME", activation=activation_l4
    ),
]

#-- Optimiser
# default: 0.001
opt_learning_rate = 0.0001  #@param{type:"raw"}
# default: 0.5
opt_beta = 0.4 #@param{type:"raw"}
gen_optimizer = tf.keras.optimizers.Adam(opt_learning_rate, beta_1=opt_beta)

# Build the network
gen = tf.keras.Sequential(generator)
# gen.build([train_images.shape[0], 1, 1, seed_size])  # specifies input shape
gen.build([4000, 1, 1, seed_size]) 
# Print network summary
# gen.summary()

In [8]:
#@title Set up discriminator network structure

# default: "relu"
activation_l1 = "relu" #@param{type: "string"}
# default: "relu"
activation_l2 = "relu" #@param{type: "string"}

# Discriminator structure: 5 layers, Input + Convolutionx2 + Flatten + Dense
discriminator = [
    tf.keras.layers.InputLayer(input_shape=DIMS),
    tf.keras.layers.Conv2D(
        filters=seed_size/2, kernel_size=3, strides=(n_stride, n_stride), activation=activation_l1
    ),
    tf.keras.layers.Conv2D(
        filters=seed_size, kernel_size=3, strides=(n_stride, n_stride), activation=activation_l2
    ),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=1, activation=None),
]

#-- Optimiser
# default: 0.005
opt_learning_rate = 0.0001  #@param{type:"raw"}
disc_optimizer = tf.keras.optimizers.RMSprop(opt_learning_rate)

# Build the network
disc = tf.keras.Sequential(discriminator)
disc.compile(loss='binary_crossentropy', optimizer= disc_optimizer , metrics = ['accuracy'])
disc.build(DIMS)  # specifies input shape

# Print network summary
# disc.summary()

In [9]:
#@title Set up the GAN as a Keras model object.

class GAN(tf.keras.Model):
    """ 
    A basic GAN class. Extends tf.keras.Model
    """

    def __init__(self, **kwargs):
        super(GAN, self).__init__()
        self.__dict__.update(kwargs)

        self.gen = self.gen
        self.disc = self.disc

    def call(self, x):
        return self.gen(x)

    def generate(self, z):
        """
        Run input vector z through the generator to create fake data.
        """
        return self.gen(z)

    def discriminate(self, x):
        """
        Run data through the discriminator to label it as real or fake.
        """
        return self.disc(x)

    def compute_loss(self, x):
        """ 
        Passes through the network and computes loss for given data.
        """
        # Generate a random vector seed from a uniform distribution
        seed = tf.random.normal([x.shape[0], 1, 1, self.seed_size])

        # Use the seed to generate a fake data set with the generator network.
        fakes = self.generate(seed)

        # Use the discriminator network to obtain labels for both the generated data (x_gen) and the real data (x)
        logits_reals = self.discriminate(x)
        logits_fakes = self.discriminate(fakes)

        # Discriminator loss, looking at correctly labeled data
        # Losses of the real data with correct label "1"
        disc_real_loss = gan_loss(logits=logits_reals, is_real=True)
        # Losses of the fake data with correct label "0"
        disc_fake_loss = gan_loss(logits=logits_fakes, is_real=False)
        # The discriminator loss is the sum of the 2 previous values
        disc_loss = disc_fake_loss + disc_real_loss

        # Generator loss, looking at the fake data labeled as real ("1")
        gen_loss = gan_loss(logits=logits_fakes, is_real=True)

        # Return losses
        return disc_loss, gen_loss

    def compute_gradients(self, x):
        """ 
        Passes through the network and computes gradients.
        """
        ### Pass x through network and compute losses
        with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
            disc_loss, gen_loss = self.compute_loss(x)

        # Compute gradients
        gen_gradients = gen_tape.gradient(gen_loss, self.gen.trainable_variables)
        disc_gradients = disc_tape.gradient(disc_loss, self.disc.trainable_variables)

        return gen_gradients, disc_gradients

    def apply_gradients(self, gen_gradients, disc_gradients):
        """
        Apply given gradients to both networks.
        """
        self.gen_optimizer.apply_gradients(zip(gen_gradients, self.gen.trainable_variables))
        self.disc_optimizer.apply_gradients(zip(disc_gradients, self.disc.trainable_variables))

    @tf.function
    def train(self, train_x):
        """
        Train the GAN! 
        """
        gen_gradients, disc_gradients = self.compute_gradients(train_x)
        self.apply_gradients(gen_gradients, disc_gradients)
        
        
def gan_loss(logits, is_real=True):
    """
    Computes cross entropy between logits and labels
    """
    if is_real:
        labels = tf.ones_like(logits)
    else:
        labels = tf.zeros_like(logits)

    # Returns loss calculation
    return tf.nn.sigmoid_cross_entropy_with_logits(labels, logits)


# Set up the model

model = GAN(
    gen = gen,
    disc = disc,
    gen_optimizer = gen_optimizer,
    disc_optimizer = disc_optimizer,
    seed_size = seed_size
)

In [10]:
#@title VGG model choice (style transfer)
# network_choice = "vgg19" #@param['vgg19', 'vgg16']

# switcher1 = {
#     "vgg19": applications.vgg19,
#     "vgg16": applications.vgg16,
# }
# switcher2 = {
#     "vgg19": applications.vgg19.VGG19(weights="imagenet", include_top=False),
#     "vgg16": applications.vgg16.VGG16(weights="imagenet", include_top=False),
# }
# network = switcher1[network_choice]

# def build_model():
#     model = switcher2[network_choice]
#     model.summary()
#     return model

In [11]:
#@title Display images function (style transfer)

def display_images(content_image, style_image, combination_image, num):
    """
    Displays content, style and combination images, labelled. Any could be missing if set to None
    """
    _, cells = plt.subplots(1, 3, figsize=(25,10))
    if content_image is not None:
        cells[0].imshow(content_image)
        cells[0].set_title("Content", fontsize=15)
    if style_image is not None:
        cells[1].imshow(style_image)
        cells[1].set_title("Style", fontsize=15)
    if combination_image is not None:
        cells[2].imshow(combination_image)
        cells[2].set_title("Artefact {}".format(num), fontsize=15)
    for cell in cells:
        cell.axis('off')
    plt.show()

In [12]:
#@title Image pre-processing (style transfer)
# content_image = keras.preprocessing.image.load_img('/content/gdrive/My Drive/ECS7022P/media/images/content.png')
# style_image = keras.preprocessing.image.load_img('/content/gdrive/My Drive/ECS7022P/media/images/style2.png')
# # Turn images into Numpy arrays to extract full dimensions
# content_image_np = keras.preprocessing.image.img_to_array(content_image)
# style_image_np = keras.preprocessing.image.img_to_array(style_image)

# # Setting image dimensions variables
# height = content_image_np.shape[0]
# width = content_image_np.shape[1]
# channels = content_image_np.shape[2]
# # Set height
# img_height = 400  #@param{type: 'number'}
# # Scale width appropriately
# img_width = int(width * img_height / height) 
# img_size = img_height * img_width

# def preprocess_image(image_path):
#     """
#     Open, resize and format pictures into appropriate tensors
#     """
#     # Load image with given size into PIL format
#     img = keras.preprocessing.image.load_img(
#         image_path, target_size=(img_height,img_width)
#     )  
#     # Turn image into Numpy array
#     img = keras.preprocessing.image.img_to_array(img)
#     # The next step expects a batch of images, so add another dimension
#     img = np.expand_dims(img, axis=0)  
#     # Call the network preprocessing step. 
#     # Converts to BGR and zero-centers data with regards to the Imagenet dataset, by subtracting the mean channel values in Imagenet
#     img = network.preprocess_input(img)  
#     # Transform to tensor
#     return tf.convert_to_tensor(img)  

# def deprocess_image(tensor):
#     """
#     Reverse the preprocessing step to turn the tensor into an RGB Numpy array which we can visualise
#     """
#     # Transform the tensor into a Numpy array
#     img = tensor.numpy()[0]

#     # Return BGR values to normal (non-zero-centered), adding back in the means of the Imagenet dataset
#     img[:, :, 0] += 103.939
#     img[:, :, 1] += 116.779
#     img[:, :, 2] += 123.68

#     # Convert from BGR to RGB
#     img = img[:, :, ::-1]

#     # Ensure values are in valid ranges
#     img = np.clip(img, 0, 255).astype("uint8")

#     return img

In [13]:
#@title Build the VGG19 model (style transfer)
# model = build_model()

# # Set up a model to extract features, given input, for each layer in the network
# outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
# feature_extractor = Model(inputs=model.inputs, outputs=outputs_dict)

# initial_learning_rate = 100.0 #@param
# decay_steps = 100 #@param
# decay_rate = 0.96 #@param

# optimizer = SGD(tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=initial_learning_rate, decay_steps=decay_steps, decay_rate=decay_rate))

# def gram_matrix(input_tensor):
#     """
#     Calculate the Gram matrix for given input
#     """
#     input_tensor = tf.transpose(input_tensor, (2, 0, 1))  # Transpose
#     features = tf.reshape(input_tensor, (tf.shape(input_tensor)[0], -1))  # Flatten layer
#     gram = tf.matmul(features, tf.transpose(features))
#     return gram

# def style_loss(style, combination):
#     """
#     The style loss is the mean square error between the Gram matrix of the style image features and the Gram matrix of the combination image features,
#     divided by 4 in original paper (see link at the end of the notebook)
#     """
#     S = gram_matrix(style)
#     C = gram_matrix(combination)
#     return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (img_size ** 2))

# def content_loss(content, combination):
#     """
#     The content loss is the mean square error between the combination and the content image features
#     """
#     return tf.reduce_sum(tf.square(combination - content))

# # Set which layers of the network should be taken into consideration when calculating content and style loss
# style_layers = [
#     "block1_conv1",
#     "block2_conv1",
#     "block3_conv1",
#     "block4_conv1",
#     "block5_conv1",
# ]
# content_layers = ["block5_conv2"]

# # Total loss is a weighted sum of content and style loss
# content_weight = 1e-8  #@param
# # style_weight = 1e-6
# style_weight = 5e-6  #@param
# content_weight /= len(content_layers)
# style_weight /= len(style_layers)

# def loss_function(combination_image, content_image, style_image):
#     """
#     Calculate the loss function given a content image, a style image and the combination result
#     """
#     # Combine all the images in the same tensor
#     input_tensor = tf.concat(
#         [content_image, style_image, combination_image], axis=0
#     )

#     # Get the features in all the layers for the three images
#     features = feature_extractor(input_tensor)

#     # Initialise the loss
#     loss = tf.zeros(shape=())

#     # Extract the content layers and calculate the content loss
#     for layer_name in content_layers:
#         layer_features = features[layer_name]
#         content_image_features = layer_features[0, :, :, :]  # Content image is at position 0
#         combination_image_features = layer_features[2, :, :, :]  # Combination image is at position 2
#         loss += content_weight * content_loss(content_image_features, combination_image_features)

#     # Extract the style layers and calculate the style loss
#     for layer_name in style_layers:
#         layer_features = features[layer_name]
#         style_image_features = layer_features[1, :, :, :]  # Style image is at position 1
#         combination_image_features = layer_features[2, :, :, :]
#         loss += style_weight * style_loss(style_image_features, combination_image_features)

#     return loss

# def compute_loss_and_grads(content_image, style_image, combination_image):
#     """
#     Brings together the calculation of loss and that of gradients
#     """
#     with tf.GradientTape() as tape:
#         loss = loss_function(combination_image, content_image, style_image)
#         grads = tape.gradient(loss, combination_image)
#     return loss, grads

# # Compile the model
# model.compile(optimizer, loss_function)


In [14]:
#@title Saving function (style transfer)

# from google.colab import drive
# drive.mount('/content/gdrive')

# # Directory where the checkpoints will be saved
# path = '/content/gdrive/My Drive/ECS7022P/models/' #@param{type: 'string'}

# def save(epoch, img_tensor, save_image=True, save_model=False):
#   # model_name = path + "model_" + str(epoch) + '.tf'
#   model_name = path + "ST" + '.tf'
#   # model_name = 'ST' + '.tf'
#   image_name = path + "img_" + str(epoch) + '.png'

#   # Save image
#   if save_image:
#       img = deprocess_image(img_tensor)
#       keras.preprocessing.image.save_img(image_name, img)

#   # Save model
#   if save_model:
#       # tf.saved_model.save(model, model_name)
#       tf.saved_model.save(model,model_name)

## Training Code:

In [15]:
#@title GAN traning

# # default: 50
# n_epochs =  800# @param{type: "integer"} 

# losses = pd.DataFrame(columns = ['disc_loss', 'gen_loss'])

# for epoch in range(n_epochs):

#     # Train the model
#     for batch, train_x in tqdm(zip(range(N_TRAIN_BATCHES), train_dataset), total=N_TRAIN_BATCHES):
#       model.train(train_x)

#     # At this point we could also test the model, compute losses and display them during training (if we'd kept training and test images separate)
#     loss = []
#     # for batch, test_x in tqdm(zip(range(N_TEST_BATCHES), test_dataset), total=N_TEST_BATCHES):
#     loss.append(model.compute_loss(train_x))
#     a = np.mean(loss, axis=0)
#     a = a[:,:,0]
#     avg_1 = np.mean(a[0,:])
#     avg_2 = np.mean(a[1,:])
#     losses.loc[len(losses)] = [avg_2, avg_1]
#     # losses.loc[len(losses)] = np.mean(loss)

#     # Display epoch and images generate at this point in training
#     display.clear_output()  # This line clears output between epochs
#     print(
#         "Epoch: {} | disc_loss: {} | gen_loss: {}".format(
#             epoch+1, losses.disc_loss.values[-1], losses.gen_loss.values[-1]
#         )
#     )
#     print("Epoch: {}".format(epoch+1))
#     generated_images = model.gen(tf.random.normal(shape=(10, seed_size)))

#     # save the generated images at the 20th, 100th, 250th, and 500th epoch
#     if int(epoch+1)==20 or int(epoch+1)==100 or int(epoch+1)==250 or int(epoch+1)==500 or int(epoch+1)==750: 
#       for i in range(9):
#         plt.subplot(3,3,i+1)
#         plt.imshow(generated_images[i,:,:,:])
#         plt.axis('off')
#         plt.savefig('/content/gdrive/My Drive/ECS7022P/media/outputs/GAN_image/epoch:{}.png'.format(epoch+1))  # store the images 
#       plt.show()
#       display.clear_output() 
    
#     show_images(generated_images)

In [16]:
#@title Display the loss curve during the GAN training process
# plt.plot(losses['disc_loss'], label='disc_loss')
# plt.plot(losses['gen_loss'], label='gen_loss')
# plt.title('the loss curve')
# plt.legend()
# plt.savefig('/content/gdrive/My Drive/ECS7022P/media/outputs/GAN_image/loss_cruve')  # store the images 
# plt.show()

In [17]:
#@title Save GAN training model to GDrive

# from google.colab import drive
# drive.mount('/content/gdrive')

# gen_save_name = 'gen' #@param{type: 'string'}
# gen_save_name += '.tf'  # Set extension separately
# disc_save_name = 'disc' #@param{type: 'string'}
# disc_save_name += '.tf' # Set extension separately

# path = 'My Drive/ECS7022P/models/' #@param{type: 'string'}
# full_path_g = F"/content/gdrive/{path}{gen_save_name}" 
# full_path_d = F"/content/gdrive/{path}{disc_save_name}" 

# gen.save(full_path_g, save_format='tf')
# disc.save(full_path_d, save_format='tf')

In [18]:
#@title VGG19 training and save VGG19 training model to GDrive

# content_image_tensor = preprocess_image('/content/gdrive/My Drive/ECS7022P/media/images/content.png')
# style_image_tensor = preprocess_image('/content/gdrive/My Drive/ECS7022P/media/images/style1.png')
# combination_image_tensor = tf.Variable(preprocess_image('/content/gdrive/My Drive/ECS7022P/media/images/content.png'))

# iterations = 5000 # @param{type: "integer"} 

# for it in range(0, iterations + 1):
#     # Step 1: calculate loss and gradients
#     loss, grads = compute_loss_and_grads(content_image_tensor, style_image_tensor, combination_image_tensor)
#     # Step 2: apply gradients
#     optimizer.apply_gradients([(grads, combination_image_tensor)])
    

#     # Display images
#     # display.clear_output()  # This line clears output between iterations
#     if it % 500 == 0:
#       print("Iteration %d: loss=%.2f" % (it, loss))
#       display_images(content_image, style_image, deprocess_image(combination_image_tensor))

#     # Save image and/or model
#     if it==5000:
#       save(it, combination_image_tensor, save_image=False, save_model=True)

## Generation Code:

In [19]:
#@title * The project has been made open source on github. We can download the saved model directly from the github URL.


In [20]:
#@title Download Saved Model
!wget "https://github.com/Shao210934641/download/raw/main/models.zip" -P "/content/models_0"
!unzip "/content/models_0/models.zip" -d "/content/models"

!wget "https://github.com/Shao210934641/download/raw/main/images.zip" -P "/content/images_0"
!unzip "/content/images_0/images.zip" -d "/content/images"
display.clear_output()

In [21]:
#@title Load model and generate images

full_path_g = '/content/models/gen.tf'
full_path_d = '/content/models/disc.tf'
gen2 = tf.keras.models.load_model(full_path_g)
disc2 = tf.keras.models.load_model(full_path_d)

# Put together GAN model with the loaded networks
model = GAN(
    gen = gen2,
    disc = disc2,
    gen_optimizer = gen_optimizer,
    disc_optimizer = disc_optimizer,
    seed_size = seed_size
)

# Generate some images!
samples = model.predict(tf.random.normal(shape=(10, 1, 1, seed_size)))

# Save some of theses images for later style transfer
for i in range(9):
  plt.subplot(3,3,i+1)
  plt.imshow(samples[i,:,:,:])
  plt.axis('off')
  plt.subplots_adjust(wspace=0, hspace=0)
# plt.savefig('/content/gdrive/My Drive/ECS7022P/media/images/content.png')  # store the images 
plt.savefig('/content/content.png')
plt.show()  # display the images
display.clear_output()
print('GAN: ')
show_images(samples)
print(' ')
print(' ')
print('Style Transfer: ')

# Load into 2 variables named style_image and content_image.
content_image = keras.preprocessing.image.load_img('/content/content.png')
style_image = keras.preprocessing.image.load_img('/content/images/images/style1.png')

content_image_np = keras.preprocessing.image.img_to_array(content_image)
style_image_np = keras.preprocessing.image.img_to_array(style_image)

# Fast Neural Style Transfer with TF-Hub
hub_model = hub.load('https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2')

# Using the numpy arrays representing our images, with 1 batch dimension added, and normalized
c_img = np.expand_dims(content_image_np, axis=0)/255 
s_img = np.expand_dims(style_image_np, axis=0)/255 

# Artefact 01
stylized_image = hub_model(tf.constant(c_img), tf.constant(s_img))[0]
display_images(c_img[0], s_img[0], stylized_image[0], num='01')

# Artefact 02
style_image = keras.preprocessing.image.load_img('/content/images/images/style2.png')
style_image_np = keras.preprocessing.image.img_to_array(style_image)
s_img = np.expand_dims(style_image_np, axis=0)/255
stylized_image = hub_model(tf.constant(c_img), tf.constant(s_img))[0]
display_images(c_img[0], s_img[0], stylized_image[0], num='02')

# Artefact 03
style_image = keras.preprocessing.image.load_img('/content/images/images/style3.png')
style_image_np = keras.preprocessing.image.img_to_array(style_image)
s_img = np.expand_dims(style_image_np, axis=0)/255 
stylized_image = hub_model(tf.constant(c_img), tf.constant(s_img))[0]
display_images(c_img[0], s_img[0], stylized_image[0], num='03')


# Artefact 04
style_image = keras.preprocessing.image.load_img('/content/images/images/style4.png')
style_image_np = keras.preprocessing.image.img_to_array(style_image)
s_img = np.expand_dims(style_image_np, axis=0)/255 
stylized_image = hub_model(tf.constant(c_img), tf.constant(s_img))[0]
display_images(c_img[0], s_img[0], stylized_image[0], num='04')


# Artefact 05
style_image = keras.preprocessing.image.load_img('/content/images/images/style5.png')
style_image_np = keras.preprocessing.image.img_to_array(style_image)
s_img = np.expand_dims(style_image_np, axis=0)/255 
stylized_image = hub_model(tf.constant(c_img), tf.constant(s_img))[0]
display_images(c_img[0], s_img[0], stylized_image[0], num='05')


# Artefact 06
style_image = keras.preprocessing.image.load_img('/content/images/images/style6.png')
style_image_np = keras.preprocessing.image.img_to_array(style_image)
s_img = np.expand_dims(style_image_np, axis=0)/255 
stylized_image = hub_model(tf.constant(c_img), tf.constant(s_img))[0]
display_images(c_img[0], s_img[0], stylized_image[0], num='06')


# Artefact 07
style_image = keras.preprocessing.image.load_img('/content/images/images/style7.png')
style_image_np = keras.preprocessing.image.img_to_array(style_image)
s_img = np.expand_dims(style_image_np, axis=0)/255 
stylized_image = hub_model(tf.constant(c_img), tf.constant(s_img))[0]
display_images(c_img[0], s_img[0], stylized_image[0], num='07')


# Artefact 08
style_image = keras.preprocessing.image.load_img('/content/images/images/style8.png')
style_image_np = keras.preprocessing.image.img_to_array(style_image)
s_img = np.expand_dims(style_image_np, axis=0)/255 
stylized_image = hub_model(tf.constant(c_img), tf.constant(s_img))[0]
display_images(c_img[0], s_img[0], stylized_image[0], num='08')

Output hidden; open in https://colab.research.google.com to view.